In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
import json

def load(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    urls = [x["url"] for x in data]
    labels = [x["label"] for x in data]
    return urls, labels

urls_train, labels_train = load("url_sqli.json")


In [8]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(urls_train)

sequences = tokenizer.texts_to_sequences(urls_train)
max_len = 100

X = pad_sequences(sequences, maxlen=max_len)
y = np.array(labels_train)


In [9]:
model = Sequential([
    Embedding(
        input_dim=len(tokenizer.word_index) + 1,
        output_dim=32,
        input_length=max_len
    ),
    LSTM(64),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.fit(
    X,
    y,
    epochs=10,
    batch_size=2
)

Epoch 1/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 59s 33ms/step - accuracy: 0.9501 - loss: 0.1302
Epoch 2/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 57s 33ms/step - accuracy: 0.9962 - loss: 0.0115
Epoch 3/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 56s 32ms/step - accuracy: 1.0000 - loss: 3.7088e-05
Epoch 4/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 83s 33ms/step - accuracy: 1.0000 - loss: 1.5204e-05
Epoch 5/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 56s 33ms/step - accuracy: 1.0000 - loss: 5.4733e-06
Epoch 6/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 57s 33ms/step - accuracy: 1.0000 - loss: 2.0083e-06
Epoch 7/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 56s 33ms/step - accuracy: 1.0000 - loss: 8.3451e-07
Epoch 8/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 58s 33ms/step - accuracy: 1.0000 - loss: 3.7184e-07
Epoch 9/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 56s 33ms/step - accuracy: 1.0000 - loss: 1.5317e-07
Epoch 10/10
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 83s 33ms/step - accuracy: 1.0000 - loss: 5.9040e-08


In [12]:
def predict_sqli(url):
    seq = tokenizer.texts_to_sequences([url])
    padded = pad_sequences(seq, maxlen=max_len)
    prob = model.predict(padded)[0][0]
    return prob, "SQL Injection" if prob > 0.5 else "Normal"

In [13]:
predict_sqli("/test.php?id=1 UNION SELECT * FROM users")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


(np.float32(0.99999976), 'SQL Injection')

In [16]:
predict_sqli("/rest/products/search?q=')) UNION SELECT 1,2,3,4,5,6,7,8,sql FROM sqlite_schema--")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


(np.float32(1.0), 'SQL Injection')

In [15]:
predict_sqli("/rest/track-order/%27%20%7C%7C%20true%20%7C%7C%20%27")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


(np.float32(0.9999995), 'SQL Injection')

In [17]:
predict_sqli("/@shahpratham529/top-100-sql-injection-payloads-complete-list-for-2025-9f9ba2d2790a")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


(np.float32(3.4393373e-08), 'Normal')

In [18]:
model.save("url_sqli_model.keras")